In [1]:
%reload_ext autoreload
%autoreload 2

import time
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import optim

from utils import IMDBDataset, SignSGD
from models.SentimentClassifier import SentimentClassifier

/Users/teframartin/EPFL_Cours/MA4/OptiMlForever/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/teframartin/EPFL_Cours/MA4/OptiMlForever/.venv/lib/python3.10/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
# initialize device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using {} device".format(device))

Using mps device


In [3]:
directory = "datasets"
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### IMDB dataset
train_imdb = IMDBDataset(root=directory, tokenizer=tokenizer, train=True)
test_imdb = IMDBDataset(root=directory, tokenizer=tokenizer, train=False)

# print(type(train_imdb))
# use data loader to load data in batches
train_dataloader_imdb = DataLoader(train_imdb, batch_size=64, shuffle=True)
test_dataloader_imdb = DataLoader(test_imdb, batch_size=64, shuffle=True)

In [5]:
print(type(train_dataloader_imdb))

<class 'torch.utils.data.dataloader.DataLoader'>


In [6]:
# get lenght of train_dataloader_imdb
print(len(train_dataloader_imdb))

626


In [4]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

# Sample data
texts = ['I love this place', 'I hate this place', 'I love dogs', 'I hate cats']
labels = [1, 0, 1, 0]  # 1 for positive sentiment, 0 for negative sentiment

# Preprocessing
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=5)

# Model
model = Sequential()
model.add(Embedding(100, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
model.fit(data, labels, epochs=10)


ValueError: Unrecognized data type: x=[[0 1 2 3 4]
 [0 1 5 3 4]
 [0 0 1 2 6]
 [0 0 1 5 7]] (of type <class 'numpy.ndarray'>)

In [9]:
pip install Keras-Preprocessing

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Create the sentiment classifier
model = SentimentClassifier(num_classes=2)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)

# Train the model
for epoch in range(10):
    print(f"Epoch {epoch+1}")
    for batch in train_dataloader_imdb:
        # Forward pass
        logits = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        # Compute the loss
        loss = loss_fn(logits, batch['label'])
        
        print(f"Loss: {loss.item()}")

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()
        optimizer.zero_grad()


Epoch 1
Loss: 0.6815488338470459


KeyboardInterrupt: 

In [ ]:
from utils import train

model = SentimentClassifier(input_dim=1000)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train(model, optimizer, train_dataloader_imdb, epochs=5, device=device)

/Users/teframartin/EPFL_Cours/MA4/OptiMlForever/utils.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
  


KeyError: 27086

In [ ]:
def train_with_optimizer(model, optimizer_type, train_loader, num_epochs=10):
    
    if optimizer_type == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    elif optimizer_type == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=0.001)
    elif optimizer_type == 'signsgd':
        optimizer = SignSGD(model.parameters(), lr=0.001)
    else:
        raise ValueError("Optimizer type not supported.")
    
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    print(f"Training with {optimizer_type}")
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    
    losses = []
    num_epochs = 10
    # loss per epoch
    start_time = time.time()

    epoch_times = []
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}")
        start_time_epoch = time.time()
        
        epoch_losses = []
        for i, (reviews, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(reviews)
            loss = criterion(outputs.squeeze(), labels)
            epoch_losses += [loss.item()]
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            
        losses.append(sum(epoch_losses) / len(epoch_losses))
        
        scheduler.step()
        
        end_time_epoch = time.time()
        
        epoch_times.append(end_time_epoch - start_time_epoch)

    end_time = time.time()
    duration = end_time - start_time
    avg_epoch_time = sum(epoch_times) / len(epoch_times)
    
    return model, losses, avg_epoch_time

In [ ]:
model = SentimentClassifier(input_dim=1000)

optimizers = ["adam", "sgd", "signsgd", "adagrad"]

losses_per_optimizer = {}
execution_times_per_optimizer = {}
accuracies_per_optimizer = {}

for optimizer in optimizers:
    model, losses, avg_epoch_time = train_with_optimizer(model, optimizer, train_dataloader_imdb)
    
    losses_per_optimizer[optimizer] = losses
    execution_times_per_optimizer[optimizer] = avg_epoch_time
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for reviews, labels in test_loader:
            outputs = model(reviews)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Accuracy of the network on the {total} test reviews: {100 * correct / total} %')
    
    accuracies_per_optimizer[optimizer] = 100 * correct / total

Training with adam
Epoch 1


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object